Transformer 이해하기

Query, Key, Value 직관적으로 이해하기

📖 1-1. 현실에서의 예시

	📌 비유: 도서관에서 책을 찾는 과정
		•	Query (질의) = 사용자가 찾고 싶은 키워드 (“딥러닝 책”)
	•	Key (키) = 도서관의 책 제목들
	•	Value (값) = 해당하는 책의 실제 내용

과정:
	1.	사용자가 “딥러닝 책”을 찾고 싶음 (Query)
	2.	도서관 시스템이 책 제목(Key)과 Query를 비교하여 가장 관련 있는 책을 찾음
	3.	해당하는 책의 내용(Value)을 제공

💡 결론: Query와 Key를 비교하여 가장 관련 있는 Value를 가져오는 것이 어텐션 메커니즘이다!

In [2]:
import torch

# Query, Key, Value를 정의
Q = torch.tensor([[1.0, 0.0, 1.0]])  # (1, 3) 크기의 행렬
K = torch.tensor([[1.0, 0.0, 1.0], 
                  [0.0, 1.0, 1.0], 
                  [1.0, 1.0, 0.0]])  # (3, 3)
V = torch.tensor([[10.0, 0.0], 
                  [0.0, 10.0], 
                  [5.0, 5.0]])  # (3, 2)

In [3]:
scores = torch.matmul(Q, K.T)
print(scores)

tensor([[2., 1., 1.]])


In [7]:
import torch.nn.functional as F

attention_weights = F.softmax(scores, dim=-1)
print(attention_weights)

tensor([[0.5761, 0.2119, 0.2119]])


In [8]:
output = torch.matmul(attention_weights, V)
print(output)

tensor([[6.8209, 3.1791]])


In [13]:
import torch.nn as nn

import torch

# 예제: 각 단어가 4차원 벡터로 표현됨 (d_model=4)
x1 = torch.tensor([1.0, 0.0, 1.0, 0.5])  # "나는"
x2 = torch.tensor([0.5, 1.0, 0.0, 1.0])  # "사과"
x3 = torch.tensor([1.0, 1.0, 0.5, 0.0])  # "를"
x4 = torch.tensor([0.0, 1.0, 1.0, 0.5])  # "먹었다"

X = torch.stack([x1, x2, x3, x4])  # (4, 4)
# print("입력 임베딩 (X):\n", X)

# 가중치 행렬 정의 (4x4 크기, 학습 가능)
W_q = nn.Linear(4, 4)  # W_query
W_k = nn.Linear(4, 4)  # W_key
W_v = nn.Linear(4, 4)  # W_value

# Query, Key, Value 생성
Q = W_q(X)
K = W_k(X)
V = W_v(X)

# print("Query (Q):\n", Q)
# print("Key (K):\n", K)
# print("Value (V):\n", V)

Parameter containing:
tensor([[-0.0776, -0.3397,  0.3057, -0.3662],
        [-0.3474,  0.4001, -0.3288,  0.0862],
        [ 0.3752,  0.4758, -0.3547, -0.0924],
        [ 0.2171, -0.1210, -0.2775, -0.2962]], requires_grad=True)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. 일상 비유 시각화: 도서관에서 책 찾기
print("===== 도서관에서 책 찾기 비유 =====")
print("Query (질문): '인공지능에 관한 책 있나요?'")
print("Keys (책 제목들):")
books = ["파이썬 기초", "인공지능 입문", "웹 개발 기초", "인공지능과 딥러닝", "자료구조"]
for i, book in enumerate(books):
    print(f"  Key {i+1}: '{book}'")

# 유사도 계산 (간단히 단어 매칭으로)
similarity = []
query = "인공지능"
for book in books:
    # 간단한 유사도: 제목에 '인공지능'이 포함되면 높은 점수
    score = 1.0 if query in book else 0.1
    similarity.append(score)

print("\n유사도 점수:")
for i, (book, score) in enumerate(zip(books, similarity)):
    print(f"  '{book}': {score:.1f}")

# 소프트맥스로 가중치 변환
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / exp_x.sum()

weights = softmax(similarity)
print("\n가중치 (소프트맥스 적용):")
for i, (book, weight) in enumerate(zip(books, weights)):
    print(f"  '{book}': {weight:.2f}")

# Values (책 내용 - 간략히)
values = [
    "파이썬 문법과 기초 개념",
    "인공지능의 개념과 역사",
    "HTML, CSS, JavaScript 기초",
    "딥러닝 알고리즘과 신경망 구조",
    "알고리즘과 데이터 구조 개념"
]

# 가중 평균으로 최종 응답 계산
response = ""
for weight, value in zip(weights, values):
    response += f"{weight:.2f} × '{value}'\n"

print("\n가중 평균된 응답 (어텐션의 결과):")
print(response)

# 시각화
plt.figure(figsize=(10, 4))
plt.bar(books, weights, color='skyblue')
plt.title('Query "인공지능"에 대한 각 Key의 가중치')
plt.ylabel('가중치')
plt.ylim(0, 0.7)
for i, w in enumerate(weights):
    plt.text(i, w + 0.02, f'{w:.2f}', ha='center')
plt.tight_layout()

# 2. 실제 벡터로 구현하는 간단한 셀프 어텐션
print("\n\n===== 벡터로 구현하는 간단한 셀프 어텐션 =====")

# 간단한 문장의 단어 벡터 (가상의 2차원 임베딩)
words = ["나는", "인공지능을", "공부한다"]
word_vectors = np.array([
    [1, 1],    # "나는"
    [2, 5],    # "인공지능을" 
    [1, 3]     # "공부한다"
])

print("단어:", words)
print("단어 벡터:\n", word_vectors)

# 간단한 가중치 행렬 (실제로는 학습됨)
np.random.seed(42)  # 재현성을 위한 시드 설정
W_query = np.array([[1, 0], [0, 1]])   # 간단히 항등 행렬로 시작
W_key = np.array([[1.2, 0.1], [0.1, 0.9]])
W_value = np.array([[0.8, 0.3], [0.2, 1.1]])

# Q, K, V 계산
Q = np.dot(word_vectors, W_query)
K = np.dot(word_vectors, W_key)
V = np.dot(word_vectors, W_value)

print("\nQuery 벡터:")
for word, q in zip(words, Q):
    print(f"  {word}: {q}")

print("\nKey 벡터:")
for word, k in zip(words, K):
    print(f"  {word}: {k}")

print("\nValue 벡터:")
for word, v in zip(words, V):
    print(f"  {word}: {v}")

# 어텐션 스코어 계산 (Q·K^T)
scores = np.dot(Q, K.T)
print("\n어텐션 스코어:")
for i, word in enumerate(words):
    print(f"  {word}:")
    for j, other_word in enumerate(words):
        print(f"    -> {other_word}: {scores[i, j]:.2f}")

# 스케일링
d_k = K.shape[1]  # key의 차원
scaled_scores = scores / np.sqrt(d_k)
print("\n스케일링된 어텐션 스코어 (√d_k={:.2f}로 나눔):".format(np.sqrt(d_k)))
for i, word in enumerate(words):
    print(f"  {word}:")
    for j, other_word in enumerate(words):
        print(f"    -> {other_word}: {scaled_scores[i, j]:.2f}")

# 소프트맥스 적용
attention_weights = softmax(scaled_scores)
print("\n어텐션 가중치 (소프트맥스 적용):")
for i, word in enumerate(words):
    print(f"  {word}:")
    for j, other_word in enumerate(words):
        print(f"    -> {other_word}: {attention_weights[i, j]:.2f}")

# 가중치와 Value 행렬 곱
output = np.dot(attention_weights, V)
print("\n최종 출력:")
for word, o in zip(words, output):
    print(f"  {word}: {o}")

# 시각화 - 어텐션 맵
plt.figure(figsize=(7, 5))
plt.imshow(attention_weights, cmap='Blues')
plt.colorbar()
plt.xticks(np.arange(len(words)), words)
plt.yticks(np.arange(len(words)), words)
plt.title('셀프 어텐션 가중치')

for i in range(len(words)):
    for j in range(len(words)):
        plt.text(j, i, f'{attention_weights[i, j]:.2f}', 
                 ha='center', va='center', 
                 color='white' if attention_weights[i, j] > 0.5 else 'black')

plt.tight_layout()

# 3. 실제 트랜스포머에서의 멀티헤드 어텐션 개념 설명
print("\n\n===== 멀티헤드 어텐션의 의미 =====")
print("멀티헤드는 여러 관점으로 데이터를 보는 것과 같습니다:")
print("- 헤드 1: 문법적 관계에 집중")
print("- 헤드 2: 의미적 관계에 집중")
print("- 헤드 3: 주제별 연관성에 집중")
print("- 등등...")
print("\n서로 다른 헤드의 결과를 합쳐서 더 풍부한 표현을 얻을 수 있습니다.")
